# Network Update 24_4

## Whats new? 

Its that time of the year again, lets see how many things needs to be changed this time.

In [2]:
from psycopg2 import connect
from psycopg2 import sql
from psycopg2.extras import execute_values
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
import configparser
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)
connection_engine = create_engine(
    URL.create(
        drivername = "postgresql",
        host = CONFIG['DBSETTINGS']['host'],
        database = CONFIG['DBSETTINGS']['database'],
        username = CONFIG['DBSETTINGS']['user'],
        password = CONFIG['DBSETTINGS']['password']))
cur = connection_engine.raw_connection().cursor()

### Number of nodes that needs to be updated
only 3!

In [12]:
sql = '''
select node_id::bigint from congestion.network_nodes_23
except 
select node_id::bigint  from here.routing_nodes_24_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

      node_id
0    30321266
1  1182267418
2    30321268


Which would seem to be nice....but now we know that they can re-use node_id, and shift their geom. 
SO actually 244 nodes shifted. Furthest shift is 110m, is it even the same node?????? Checked, its on the 401.

In [20]:
sql = '''

select distinct node_id::bigint, (ST_transform(retired_nodes.geom, 2952) <-> ST_Transform(z.geom, 2952)) 
from congestion.network_nodes_23 retired_nodes
inner join here.routing_nodes_24_4 z using (node_id) 
where (ST_transform(retired_nodes.geom, 2952) <-> ST_Transform(z.geom, 2952)) >0
order by (ST_transform(retired_nodes.geom, 2952) <-> ST_Transform(z.geom, 2952)) desc

    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

      node_id    ?column?
0    30456075  110.281672
1    30323391   85.815058
2    30445533   85.162731
3    30371548   75.761128
4    30436212   71.470214
..        ...         ...
239  30444869    0.805097
240  30444879    0.805094
241  30353366    0.805037
242  30353367    0.805037
243  30447104    0.805005

[244 rows x 2 columns]


# Number of links that needs to be updated
871 links needs to be retired

In [13]:
sql = '''
select link_dir from congestion.network_links_23_4
except 
select link_dir from here.routing_streets_24_4 
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

        link_dir
0      29572498T
1     993183732F
2     822382890T
3    1074945293F
4     806101976T
..           ...
866   995779172F
867   133790136T
868   792244130T
869   799979707T
870   845742118F

[871 rows x 1 columns]


Again, tricky tricky, because they reuse links. The biggest change is 247m, like, that really should just be a link_dir. Anyways, they are all the highways, with just a few exception.

In [24]:
sql = '''
select link_dir, 
    ST_length(ST_transform(retired_nodes.geom, 2952)) - ST_length(ST_transform(z.geom, 2952))::int
    
from congestion.network_links_23_4 AS retired_nodes
inner join here.routing_streets_24_4 z using (link_dir) 
where ST_length(ST_transform(retired_nodes.geom, 2952)) - ST_length(ST_transform(z.geom, 2952))  >0
order by abs(ST_length(ST_transform(retired_nodes.geom, 2952)) - ST_length(ST_transform(z.geom, 2952)) ) desc
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

         link_dir    ?column?
0      815913053F  247.285026
1     1230329545T  171.332578
2       29602707F  146.720751
3     1231209840F  143.124859
4      995662230F  121.552443
...           ...         ...
1441  1258670222T    0.286418
1442   996840287T    0.200724
1443   996840287F    0.200724
1444   811393658F   -0.350491
1445   811393658T   -0.350491

[1446 rows x 2 columns]


### Number of segments that needs to be updated
472 segment_ids needs be to retired and route.....For ones with different link_dirs.

In [14]:
sql = '''
with changed_links AS (
select link_dir from congestion.network_links_23_4
except 
select link_dir from here.routing_streets_24_4 )
	
select distinct segment_id from  congestion.network_links_23_4 
inner join changed_links using (link_dir)
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

     segment_id
0          5843
1          1589
2           764
3          3834
4          5644
..          ...
467        4833
468        6341
469        2787
470        3221
471        2850

[472 rows x 1 columns]


including links with changed link_dir, there are 1097 segments that needs to be updated.

In [26]:
sql = '''
with changed_links AS (
select link_dir from congestion.network_links_23_4
except 
select link_dir from here.routing_streets_24_4 )
	
select distinct segment_id from  congestion.network_links_23_4 
inner join changed_links using (link_dir)
union
select distinct segment_id
from congestion.network_links_23_4 AS retired_nodes
inner join here.routing_streets_24_4 z using (link_dir) 
where ST_length(ST_transform(retired_nodes.geom, 2952)) - ST_length(ST_transform(z.geom, 2952))  >0


    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
print(nodes)   

      segment_id
0           6114
1            273
2           4326
3           2466
4            292
...          ...
1092        6341
1093        3565
1094        7004
1095        4035
1096        2850

[1097 rows x 1 columns]


### Number of new and retired traffic signals
Quite a bunch of new traffic signals. We can deal with this after we recreated the network. 

In [16]:
sql = '''
select count(1) from gis.traffic_signal
where activationdate  >= '2024-01-01'
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()   

,count
0,37


## Updating the nodes


It does return good matched, but there is one on lakeshore which i will have to double check to make sure its correct.

In [19]:
sql = '''
with retired_nodes AS (
	select distinct node_id, geom 
	from congestion.network_nodes_23
	inner join (
        select node_id::bigint from congestion.network_nodes_23
        except 
        select node_id::bigint  from here.routing_nodes_24_4 )a using (node_id))
				
select 	retired_nodes.node_id::int as old_id, nodes.node_id::int  as new_id, geom, node_geom, dist
from 	retired_nodes
CROSS JOIN LATERAL (SELECT z.node_id,
							z.geom AS node_geom, 
							(ST_transform(retired_nodes.geom, 2952) <-> ST_Transform(z.geom, 2952)) as dist	
					FROM here.routing_nodes_24_4 z
					ORDER BY (retired_nodes.geom <-> z.geom)
					LIMIT 1) nodes;
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()   

,old_id,new_id,geom,node_geom,dist
0,30321266,1258420520,0101000020E6100000C8EF6DFAB3E753C0B21188D7F5DB...,0101000020E6100000B6847CD0B3E753C0D5E76A2BF6DB...,1.372401
1,30321268,1258420518,0101000020E61000005D6DC5FEB2E753C02BDEC83CF2DB...,0101000020E61000003997E2AAB2E753C04EB4AB90F2DB...,1.957362
2,1182267418,878395939,0101000020E61000008FFCC1C073DD53C0C898BB9690DF...,0101000020E6100000A167B3EA73DD53C0CFBD874B8EDF...,7.818326


### Creating new node table

New step to use new node's geom.

```sql
CREATE TABLE congestion.network_nodes_24_4 AS
select distinct node_id::bigint, z.geom
from congestion.network_nodes_23 retired_nodes
inner join here.routing_nodes_24_4 z using (node_id) 
```

# Updating the links

Create new network_links_23_4 table with newly routed segments for ones that contains outdated links

```sql
-- Create new links table except the segment_ids that are outdated

CREATE TABLE congestion.network_links_24_4 AS 
-- retired links
with changed_links AS (
	select link_dir from congestion.network_links_23_4
	except 
	select link_dir from here.routing_streets_24_4)
-- retired segments	
, changed_seg AS (
	select distinct segment_id, start_vid, end_vid 
	from  congestion.network_links_23_4
	inner join changed_links using (link_dir))
-- everything in network_links other than the retired segments
select      a.segment_id, a.start_vid, a.end_vid, routing_streets_24_4.link_dir, routing_streets_24_4.geom, routing_streets_24_4.length
from 		congestion.network_links_23_4 a
inner join  here.routing_streets_24_4 using (link_dir)
left join 	changed_seg using (segment_id) 
where 		changed_seg is null ;

```

```sql
-- Insert routed results using new map version (24_4) for retired segments
with changed_links AS (
	select link_dir from congestion.network_links_23_4
	except 
	select link_dir from here.routing_streets_24_4)	
, changed_seg AS (
	select distinct segment_id, start_vid, end_vid 
	from  congestion.network_links_23_4
	inner join changed_links using (link_dir))
 
 -- insert result to newly created network_links table
INSERT INTO congestion.network_links_24_4
SELECT segment_id, start_vid, end_vid, link_dir, routing.geom, round(st_length(st_transform(geom, 2952))::numeric, 2) as length
FROM   changed_seg
CROSS JOIN LATERAL pgr_dijkstra('SELECT id, source::int, target::int, st_length(st_transform(geom, 2952)) as cost 
				 	   			FROM here.routing_streets_24_4',  -- route using new map version's routing_streets
								start_vid, 
								end_vid)
INNER JOIN here.routing_streets_24_4 routing ON id = edge;
```

### Check to see if all segments are inserted 
Returns a couple. 

In [27]:
sql = '''
select distinct segment_id from congestion.network_links_23_4
except
select distinct segment_id from congestion.network_links_24_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,segment_id
0,6148
1,3747
2,3723
3,6237
4,3173


### Check to see if both returned the same number of segment_id
Both have 6558 segments. 

In [30]:
sql = '''

select count(distinct segment_id) from congestion.network_links_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6580


In [31]:
sql = '''
select count(distinct segment_id) from congestion.network_links_24_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6574


## Find if there are new traffic signals to add in 
25?!?!
Check them out on QGIS and see if they are legit. 
Checked: They are mostly legit other than one that are at a midblock and seems like a peds crossing.

In [36]:
sql = '''
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as geom 
	from gis.traffic_signal
	left join congestion.network_int_px_22_2 a using (px)
	where activationdate >= '2022-04-17' and a.px is null )

select count(distinct px)
from congestion.network_segments seg
join new_signal on ST_intersects(new_signal.geom, seg.geom)
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,25


## Before fixing those, I want to fix the ones I know that is already wrong. AKA Allen road.

realized there are a lot, so i wrote a not too clean sql to determine where the start/end link had sharp turns, and if they have sharp turns, give me the proposed new source/target.

```sql
CREATE table natalie.proposed_new_vids AS
with some_order AS (
select segment_id, start_vid, end_vid, link_dir, source, target, 
case when start_vid = source then 1
when end_vid = target then 10
else null end as orders
from congestion.network_links_24_4
inner join here.routing_streets_24_4 using (link_dir)
order by segment_id)

, first_n_last_link AS (
select segment_id, start_vid, end_vid, link_dir, source, target, orders
from some_order
where orders is not null
union 
select f_l.segment_id, f_l.start_vid, f_l.end_vid, f_l.link_dir, f_l.source, f_l.target, 2 AS orders
from some_order f_l
inner join (select * from some_order where orders = 1) nxt 
    on f_l.source = nxt.target and f_l.segment_id = nxt.segment_id
union
select f_l.segment_id, f_l.start_vid, f_l.end_vid, f_l.link_dir, f_l.source, f_l.target, 9 AS orders
from some_order f_l
inner join (select * from some_order where orders = 10) nxt 
    on f_l.target = nxt.source and f_l.segment_id = nxt.segment_id
where f_l.orders is null
order by segment_id, orders)
, with_dir AS (
SELECT segment_id, start_vid, end_vid, a.source, a.target, gis.direction_from_line(a.geom) dir, orders, a.geom
from first_n_last_link
inner join here.routing_streets_24_4 a using (link_dir)
order by segment_id, orders)
, results AS (
select o1.segment_id, o1.start_vid, o1.end_vid, 
case when o1.dir!= o2.dir AND o2.dir IS not null THEN o1.target ELSE null END AS proposed_source,
case when o3.dir!= o4.dir AND o4.dir IS not null THEN o4.source ELSE null END AS proposed_target,
o1.dir as o1_dir, o2.dir as o2_dir, o3.dir as o3_dir, o4.dir as o4_dir
from (select * from with_dir where orders = 1) as o1
left join (select * from with_dir where orders = 2) as o2 using (segment_id)
left join (select * from with_dir where orders = 9) as o3 using (segment_id)
left join (select * from with_dir where orders = 10) as o4 using (segment_id))

SELECT * from results
where proposed_source is not null or proposed_target is not null
```



i THOUGHT this is gonna fix more than a couple, cause the above returned 270 segments, but i guess toronto's roads are just that curvy. Checked on QGIS, it fixes about 41 of them. At least I don't have to hard code to update 41 segments??? 

now re-route them and see if they fixes things. Some of them ended up not correct still, have to manually filter but at least some is ok.

```sql
create table congestion.proposed_vid_links AS
with new_sets AS (
select segment_id, case when proposed_source is null then start_vid else proposed_source end as start_vid
, case when proposed_target is null then end_vid else proposed_target end as end_vid
from congestion.proposed_vid
inner join natalie.proposed_new_vids  using (segment_id))

SELECT segment_id, start_vid, end_vid, link_dir, routing.geom, 
    round(st_length(st_transform(geom, 2952))::numeric, 2) as length
FROM   new_sets
CROSS JOIN LATERAL pgr_dijkstra('SELECT id, source::int, target::int, st_length(st_transform(geom, 2952)) as cost 
				 	   			FROM here.routing_streets_24_4',  -- route using new map version's routing_streets
								start_vid::int, 
								end_vid::int)
INNER JOIN here.routing_streets_24_4 routing ON id = edge
order by segment_id;
```


```sql
select * From congestion.proposed_vid_links
where segment_id not in (1089,4309, 6011,7204, 298,2973,2931,3197)
```

delete wrong things in network_links and then insert the correct ones

```sql
delete from congestion.network_links_24_4 
where segment_id in (
select distinct segment_id From congestion.proposed_vid_links
where segment_id not in (1089,4309, 6011,7204, 298,2973,2931,3197));
```
```sql
insert into congestion.network_links_24_4 
select * From congestion.proposed_vid_links
where segment_id not in (1089,4309, 6011,7204, 298,2973,2931,3197)
```

because we just proposed new nodes, we will have to add them to network_nodes

```sql 
insert into congestion.network_nodes_24_4
with new_vid AS (
select  start_vid as node_id From congestion.proposed_vid_links
where segment_id not in (1089,4309, 6011,7204, 298,2973,2931,3197)
union 
select  end_vid From congestion.proposed_vid_links
where segment_id not in (1089,4309, 6011,7204, 298,2973,2931,3197)
)
select distinct b.node_id, b.geom
from new_vid
inner join here.routing_nodes_24_4 b using (node_id)
left join congestion.network_nodes_24_4 a using (node_id)
where a.node_id is null
```

## Find closest nodes for these new traffic signals 

Find new traffic signals, except the temporary / portable ones.

In [32]:
'''
-- Find segments that needs to be updated due to new traffic signals
with new_signal as (
	select px, ST_Transform(ST_buffer(ST_Transform(geom, 2952), 50), 4326) as bgeom, geom 
	from gis.traffic_signal
	left join congestion.network_int_px_23_4 a using (px)
	where activationdate >= '2023-12-18' and a.px is null )
'''

### Exclude traffic signals that should not be included

px 2617 - too far away, nearest node is 40m away

px 2650 - its on a ramp, not a part of network segments

## Route segments that was affected by new traffic signal

This query gathers all new nodes (find by nearest neighbour with traffic signal), as well as the start and end nodes of those affected segments (segments where new traffic signal lies). This creates a new table so its easier to QC on QGIS and make modifications.

In [ ]:
create table congestion.new_24_4_px_seg AS
WITH affected_nodes AS (
select distinct node_id from congestion.network_nodes_24_4
except
(select start_vid from congestion.network_segments_24_4
union 
select end_vid from congestion.network_segments_24_4
) )
, staging AS (
SELECT segment_id, link_dir, node_id as new_id, start_vid, end_vid
from congestion.network_links_24_4
inner join here.routing_streets_24_4 using (link_dir)
inner join affected_nodes on node_id = source or node_id = target
order by segment_id)

, new_segments as (
select segment_id, start_vid, new_id as end_vid
from staging
union 
select segment_id, new_id, end_vid
from staging)
, segs AS (
select 7230+row_number()over () as segment_id, start_vid, end_vid, segment_id as old_seg
from new_segments)

SELECT results.*, segment_id, link_dir, start_vid, end_vid, routing_grid.id, routing_grid.geom, old_seg
	FROM segs
	, LATERAL pgr_dijkstra('SELECT id, source::int, target::int,
						   st_length(st_transform(geom, 2952)) as cost
						   FROM here.routing_streets_24_4 routing_grid
						   ',
				start_vid, end_vid) results
	INNER JOIN here.routing_streets_24_4 routing_grid ON id = edge
order by segment_id


Delete old segment_id
```sql
with segs AS (
select distinct old_seg as segment_id 
from congestion.new_24_4_px_seg)
, delete_statement1 AS (
delete from congestion.network_links_24_4
WHERE segment_id in (select distinct segment_id from segs)
)

delete from congestion.network_segments_24_4
WHERE segment_id in (select distinct segment_id from segs)
```
Insert new one after validating to links and segment table
```sql
insert into congestion.network_links_24_4
SELECT segment_id, start_vid, end_vid, link_dir , geom, cost as length
FROM congestion.new_24_4_px_seg 
insert into congestion.network_segments_24_4
SELECT segment_id, start_vid, end_vid, ST_linemerge(ST_union(geom)) , sum(cost), false, 
gis.direction_from_line(ST_linemerge(ST_union(geom))) as dir
from  congestion.new_24_4_px_seg 
group by segment_id, start_vid, end_vid;
```
Update them to retired segment atble
```sql

insert into congestion.network_segments_retired
select  old_seg, old_v.start_Vid, old_v.end_vid, old_v.geom, old_v.total_length, old_v.highway,
		_s.int_id as start_int, _t.int_id as end_int, 
		_s.px::int start_px, _t.px::int as end_px, 
        '23_4' as here_version, '20231001' as centreline_version, 
		'2025-05-14'::date as retired_date, 'new traffic signal' as retired_reason, 
	  	 array_agg(distinct seg.segment_id) as replaced_id, 
           null::Date as valid_from, null::Date as valid_to
           
from congestion.new_24_4_px_seg seg
inner join congestion.network_segments_23_4 old_v on old_seg = old_v.segment_id
INNER JOIN congestion.network_int_px_23_4 _s on old_v.start_vid = _s.node_id -- to get equivalent start px and int_id
INNER JOIN congestion.network_int_px_23_4 _t on old_v.end_vid  = _t.node_id -- to get equivalent end px and int_id

group by old_seg,old_v.start_Vid, old_v.end_vid, old_v.geom, old_v.total_length, old_v.highway, start_int
, end_int, start_px, end_px, here_version, centreline_version, retired_date, retired_reason, valid_From, valid_to
        
```

Major changes: 
23 old segments retired. 


# Validate

## Make sure there are no duplication 

### Table `network_segments`

Total number of rows in network_segments

In [13]:
sql = '''
select count(1) 
from congestion.network_segments_24_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6612


No duplicated segment_id

In [14]:
sql = '''
select count(1) 
from (select distinct segment_id from congestion.network_segments_24_4)a
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6612


No duplicated sets of the same segment

In [15]:
sql = '''
select count(1) 
from (select distinct start_vid, end_vid from congestion.network_segments_24_4)a
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,6612


### Table `network_nodes`

Total number of nodes

In [23]:
sql = '''
select count(1) 
from congestion.network_nodes_24_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,3312


There are no duplicated node_ids

In [24]:
sql = '''
select count(1) 
from (select distinct node_id from congestion.network_nodes_24_4)a
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,3312


Are there nodes in network_segments_24 that are not in nodes table?

Returns none.

In [25]:
sql = '''
WITH all_nodes AS (
	select start_vid as node_id
	from congestion.network_segments_24_4
	union 
	select end_vid as node_id
	from congestion.network_segments_24_4)
select node_id from all_nodes
except 
select node_id from congestion.network_nodes_24_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,node_id


the other way around? also none

In [27]:
sql = '''
WITH all_nodes AS (
	select start_vid as node_id
	from congestion.network_segments_24_4
	union 
	select end_vid as node_id
	from congestion.network_segments_24_4)

select node_id from congestion.network_nodes_24_4
except
select node_id from all_nodes
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,node_id


# NATALIE come back later
Are all the nodes in here referenced in the new px<>int<>node table? 


In [91]:
sql = '''
select node_id::int from congestion.network_nodes_23
except 
select node_id::int from congestion.network_int_px_23_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,node_id
0,1050298501
1,30375021


### Table `network_links`

Total number of links

In [21]:
sql = '''

select count(1) 
from congestion.network_links_24_4
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,50974


a bunch of them are duplicate, where???
- fixed, now only 1, which is correct. 

In [22]:
sql = '''
select count(1) 
from (select distinct link_dir from congestion.network_links_24_4)a
    '''
with con : 
    nodes = pandasql.read_sql(sql, connection_engine) 
nodes.head()  

,count
0,50973
